In [1]:
import time


from simple_socket.zf_socket import SocketServer
from split_llm.common.communication import Node
from split_llm.common.communication import Communication, Node, SimulatedCommunication
from split_llm.common.real_communication import RealCommunication

device = "cuda"


# Set up communication

address_dict = {
    "127.0.0.1:9100": "n0",
    "127.0.0.1:9101": "n1",
    "127.0.0.1:9102": "n2"
}
sock0 = SocketServer("127.0.0.1:9100", address_dict, 1000)
sock1 = SocketServer("127.0.0.1:9101", address_dict, 1000)
sock2 = SocketServer("127.0.0.1:9102", address_dict, 1000)

time.sleep(1) # Wait the server to start listening

sock0.connect_all()
sock1.connect_all()
sock2.connect_all()

comm0 = RealCommunication(["n0", "n1", "n2"], {"n0": sock0}, tensor_device=device)
comm1 = RealCommunication(["n0", "n1", "n2"], {"n1": sock1}, tensor_device=device)
comm2 = RealCommunication(["n0", "n1", "n2"], {"n2": sock2}, tensor_device=device)
# comm0 = comm1 = comm2 = SimulatedCommunication(["n0", "n1", "n2"])
# comm0.new_stage("Test")
n0 = Node(comm0, "n0")
n1 = Node(comm1, "n1")
n2 = Node(comm2, "n2")

In [2]:
import os
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

/root/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.20s/it]


In [3]:
n0.space.word_embedding = glm.condgen.transformer.word_embeddings.weight.float().cuda()

In [4]:
import torch
hs = torch.normal(0, 1, [4096]).cuda()
h0 = torch.normal(0, 10, [4096]).cuda()
h1 = hs - h0

In [5]:
prediction = torch.argmax(glm.condgen.lm_head.cuda()(hs))
print("Expected prediction: ", prediction.item())

Expected prediction:  5


In [6]:
try:
    import sys
    del sys.modules['split_llm.glm6b.secure_inference']
    del sys.modules['homomorphic_encryption.bfv']
    print("delete complete!")
except Exception as e:
    print(e)
from split_llm.glm6b.secure_inference import GLM_PredictionProtocol, GLM_EmbeddingRetrievalProtocol, GLMConfig
from split_llm.common.torch_utils import relative_error

delete complete!


In [7]:
embedding_protocol0 = GLM_EmbeddingRetrievalProtocol(n0, Node.from_remote_name("n1"), Node.from_remote_name("n2"), 10, device="cuda")
embedding_protocol1 = GLM_EmbeddingRetrievalProtocol(Node.from_remote_name("n0"), n1, Node.from_remote_name("n2"), 10, device="cuda")
embedding_protocol2 = GLM_EmbeddingRetrievalProtocol(Node.from_remote_name("n0"), Node.from_remote_name("n1"), n2, 10, device="cuda")

prediction_protocol0 = GLM_PredictionProtocol(n0, Node.from_remote_name("n1"), Node.from_remote_name("n2"), embedding_protocol0.name, 10, device="cuda")
prediction_protocol1 = GLM_PredictionProtocol(Node.from_remote_name("n0"), n1, Node.from_remote_name("n2"), embedding_protocol1.name, 10, device="cuda")
prediction_protocol2 = GLM_PredictionProtocol(Node.from_remote_name("n0"), Node.from_remote_name("n1"), n2, embedding_protocol2.name, 10, device="cuda")


In [8]:
import threading


# Prepare embedding_retrieval
emb_prepare1 = threading.Thread(target=embedding_protocol1.prepare)
emb_prepare2 = threading.Thread(target=embedding_protocol2.prepare)
emb_prepare1.start()
emb_prepare2.start()
embedding_protocol0.prepare()
emb_prepare1.join()
emb_prepare2.join()

print("embedding_retrieval prepare finished")

prepare1 = threading.Thread(target=prediction_protocol1.prepare)
prepare2 = threading.Thread(target=prediction_protocol2.prepare)
prepare1.start()
prepare2.start()
prediction_protocol0.prepare()
prepare1.join()
prepare2.join()

print("prediction prepare finished")

offline1 = threading.Thread(target=prediction_protocol1.offline_execute)
offline2 = threading.Thread(target=prediction_protocol2.offline_execute)
offline1.start()
offline2.start()
prediction_protocol0.offline_execute()
offline1.join()
offline2.join()


KeyboardInterrupt: 

In [ ]:
n0.storage[f"{prediction_protocol0.name}:x0"] = h0
n1.storage[f"{prediction_protocol1.name}:x1"] = h1
online1 = threading.Thread(target=prediction_protocol1.online_execute)
online2 = threading.Thread(target=prediction_protocol2.online_execute)
online1.start()
online2.start()
prediction_protocol0.online_execute()
online1.join()
online2.join()
print("Computed prediction: ", n1.storage[f"{prediction_protocol1.name}:z"])

Computed prediction:  [94781]


In [9]:
n2.storage['GLM__EmbeddingLayer/onehot_matmul:beaver_u'].shape

torch.Size([130528, 4096])

KeyError: 'GLM__EmbeddingLayer/onehot_matmul:x-u'